In [43]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (79.6 kB/s)
Reading package li

In [44]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-26 02:09:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.46MB/s    in 0.2s    

2022-09-26 02:09:52 (5.46 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [45]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [46]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...|2015-08-31 00:00:00|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...| 

### Create DataFrames to match tables

In [47]:
from pyspark.sql.functions import to_date, count, sum
import pandas as pd
# Read in the Review dataset as a DataFrame


In [48]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|                Y|
|R3LCIANTN1H9EC|          4|            1|          1|   N|                Y|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|                Y|
|R29MB6N7HB6NZI|          1|            2|          2|   N|                Y|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|     

**For Delivery #2**

In [49]:
# total votes larger than 20
vine_df_total_20 = vine_df.filter("total_votes>20")
vine_df_total_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|                Y|
| RUBBVNQCSYCDF|          5|           23|         24|   N|     

In [50]:
# helpful_votes / total_votes > 50%
vine_df_helpful = vine_df_total_20.filter("helpful_votes/total_votes>0.5")
vine_df_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|                Y|
| RUBBVNQCSYCDF|          5|           23|         24|   N|     

In [51]:
# Vine Program = Yes
vine_df_yes = vine_df_helpful.filter("vine='Y'")
vine_df_yes.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
|R2T83WIPG3IX5D|          4|           72|         76|   Y|                N|
| RD006IWFCUA5A|          5|          122|        133|   Y|                N|
| REOQCBUVUPXP1|          4|           23|         27|   Y|                N|
|R2336R80ZEOKTX|          5|           63|         63|   Y|                N|
| RC1ZXE5GVL9TA|          4|           20|         23|   Y|     

In [52]:
# Vine Program = No
vine_df_no = vine_df_helpful.filter("vine='N'")
vine_df_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|                Y|
| RUBBVNQCSYCDF|          5|           23|         24|   N|     

In [56]:
# total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
# for vine program = yes:
total_votes_vine_yes = vine_df_yes.select(sum("total_votes"))
total_votes_vine_yes.show()

stars_5_vine_yes = vine_df_yes.filter("star_rating=5").select(sum("star_rating"))
stars_5_vine_yes.show()

percent_5_star_yes = stars_5_vine_yes.first()["sum(star_rating)"] / total_votes_vine_yes.first()["sum(total_votes)"] *100
print(percent_5_star_yes)

+----------------+
|sum(total_votes)|
+----------------+
|             932|
+----------------+

+----------------+
|sum(star_rating)|
+----------------+
|              50|
+----------------+

5.36480686695279


In [57]:
# total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
# for vine program = No:
total_votes_vine_no = vine_df_no.select(sum("total_votes"))
total_votes_vine_no.show()

stars_5_vine_no = vine_df_no.filter("star_rating=5").select(sum("star_rating"))
stars_5_vine_no.show()

percent_5_star_no = stars_5_vine_no.first()["sum(star_rating)"] / total_votes_vine_no.first()["sum(total_votes)"] *100
print(percent_5_star_no)

+----------------+
|sum(total_votes)|
+----------------+
|          380179|
+----------------+

+----------------+
|sum(star_rating)|
+----------------+
|           16325|
+----------------+

4.294029917486237
